# Python Decorators

Decorators are a powerful and expressive feature in Python that allow you to modify the behavior of functions or classes. They enable you to wrap another function to extend its behavior without permanently modifying it.

1.	Understanding Closures
2.	Simple Decorators
3.	Decorators with Callables
4.	Parameterized Decorators
5.	Common Use Cases
6.	Best Practices

## 1. Understanding Closures

Before diving into decorators, it’s essential to understand closures, as they form the foundation of how decorators work.

### 1.1. What is a Closure?

A closure is a function object that remembers values in enclosing scopes even if they are not present in memory. Closures allow the function to access those captured variables through the closure’s copies of the variables, maintaining their state between calls.

### 1.2. Example of a Closure

In [7]:
def outer_function(msg):
    message = msg  # Enclosed variable

    def inner_function():
        print(message)  # Accessing enclosed variable

    return inner_function  # Returning the inner function

# Usage
greet = outer_function("Hello, World!")
greet()  # Output: Hello, World!

Hello, World!


In [8]:
greet.__closure__[0].cell_contents

'Hello, World!'

Explanation:
- outer_function defines a variable message and an inner_function that accesses message.
- Even after outer_function has finished executing, inner_function retains access to message through the closure.

## 2. Simple Decorators

Decorators are functions that modify the behavior of other functions. A simple decorator wraps a function, enhancing or altering its behavior without changing its core functionality.

### 2.1. Creating a Simple Decorator

Let’s create a decorator that prints messages before and after the execution of a function.

In [2]:
def my_decorator(func):
    def wrapper():
        print("Before the function is called.")
        func()
        print("After the function is called.")
    return wrapper

@my_decorator
def say_hello():
    print("Hello!")

# Usage
say_hello()

Before the function is called.
Hello!
After the function is called.


### 2.2. How It Works

The @my_decorator syntax is syntactic sugar for:

```python
say_hello = my_decorator(say_hello)
```

- my_decorator takes say_hello as an argument and returns the wrapper function.
- When say_hello() is called, it actually calls wrapper(), which adds behavior before and after the original say_hello function.



## 3. Decorators with Callables

While functions are the most common callables in Python, decorators can also be applied to classes or objects that implement the __call__ method. This allows for more complex and stateful decorators.

### 3.1. Class-Based Decorators

Creating decorators as classes can be beneficial when you need to maintain state or when the decorator requires multiple parameters.

In [4]:
class CountCalls:
    def __init__(self, func):
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__}")
        return self.func(*args, **kwargs)

@CountCalls
def say_hi():
    print("Hi!")

# Usage
say_hi()
say_hi()

Call 1 of say_hi
Hi!
Call 2 of say_hi
Hi!


### 3.2. How It Works

- CountCalls is a class that takes a function func as an argument during initialization.
- The __call__ method allows the class instance to be callable.
- Each time say_hi() is called, the __call__ method increments num_calls and prints the call count before executing the original function.

### 3.3. Benefits of Class-Based Decorators
- State Maintenance: Easily keep track of state (e.g., number of calls).
- Multiple Methods: Can include additional methods or attributes as needed.
- Inheritance: Can inherit from other classes or mixins for extended functionality.

## 4. Parameterized Decorators

Sometimes, you might want your decorator to accept arguments. This requires an additional layer of function nesting, often referred to as a decorator factory.

### 4.1. Creating a Parameterized Decorator

Let’s create a decorator that repeats the execution of a function a specified number of times.

In [5]:
def repeat(num_times):
    def decorator_repeat(func):
        def wrapper(*args, **kwargs):
            for _ in range(num_times):
                func(*args, **kwargs)
        return wrapper
    return decorator_repeat

@repeat(num_times=3)
def greet(name):
    print(f"Hello, {name}!")

# Usage
greet("Alice")

Hello, Alice!
Hello, Alice!
Hello, Alice!


### 4.2. How It Works
- repeat is a decorator factory that takes num_times as an argument and returns the actual decorator decorator_repeat.
- decorator_repeat takes the function func to be decorated and returns the wrapper.
- wrapper executes func num_times times.

## 5. Common Use Cases and Best Practices
1.	Logging & Debugging: Automatically log function calls and returns.
2.	Caching (Memoization): Use decorators like functools.lru_cache to cache results of expensive functions.
3.	Validation: Validate arguments before passing them to the function.
4.	Authorization: Restrict access based on roles or permissions.
5.	Rate-Limiting: Throttle calls to certain functions (often combined with external libraries).

#### Best Practices
-	Always use functools.wraps to preserve metadata.
-	Keep decorator logic minimal and clear—avoid turning them into “black boxes.”
-	Test your decorators with different function signatures and ensure they handle special cases.

## 6. Best Practices

When writing decorators, adhering to best practices ensures that your decorators are robust, maintainable, and compatible with other decorators and Python features.

### 6.1. Use functools.wraps

The functools.wraps decorator preserves the metadata of the original function, such as its name, docstring, and module. Without it, the decorated function’s metadata would be replaced by the wrapper’s.


In [9]:
import functools

def my_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print("Decorator is running.")
        return func(*args, **kwargs)
    return wrapper

@my_decorator
def say_hello():
    """Greets the user."""
    print("Hello!")

# Usage
print(say_hello.__name__)  # Output: say_hello
print(say_hello.__doc__)   # Output: Greets the user.

say_hello
Greets the user.


In [11]:
# Without functools.wraps:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        """Wrapper docstring"""
        print("Decorator is running.")
        return func(*args, **kwargs)
    return wrapper

@my_decorator
def say_hello():
    """Greets the user."""
    print("Hello!")

print(say_hello.__name__)  # Output: wrapper
print(say_hello.__doc__)   # Output: None

wrapper
Wrapper docstring
